In [2]:
### Library Imports
import pandas as pd
import pandasql as ps


### Script Imports
import utils
import final_account
import final_card
import final_client
import final_districts
import final_loans
import final_trans


### Get all dataframes

accounts = final_account.cleanup()
card = final_card.cleanup()
clients = final_client.cleanup()
districts = final_districts.cleanup()
loans = final_loans.cleanup()
trans = final_trans.cleanup()
disp = pd.read_csv('../../project/banking_data/disp.csv', sep=';')




C:\Users\Hugo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
### Merge everything with sequel queries

### Create copy
df = loans.copy()


### Merge accounts
df = df.merge(accounts, how='left', on="account_id")
df = df.rename(columns={"date_x": "loan_date", "date_y" : "account_date"})

### Merge Disp
### Mover para cleanup
q1 = "SELECT * FROM disp WHERE Type='OWNER'"
disp = ps.sqldf(q1)
df = df.merge(disp, how='left', on="account_id")

# print(df)
### Merge Clients
clts = clients.drop(columns=['district_id'])
df = df.merge(clts, how='left', on="client_id")

### Merge Districts 
distrs = districts.rename(columns={"code ": "district_id"})
df = df.merge(distrs, how='left', on="district_id")


df.columns
### Merge transactions
trans2 = trans.rename(columns={"type": "trans_type" , "amount":"trans_ammount"})
df = df.merge(trans2, how='left', on="account_id")

# ### Merge Card
# ## TODO: Rename issue column to date_something
# card = card.rename(columns={"type": "type_card"})
# df = df.merge(card, how='left', on="disp_id")


df

### Merge

,loan_id,account_id,loan_date,amount,duration,payments,status,district_id,frequency,account_date,...,no. of commited crimes '95,no. of commited crimes '96,crime_growth,total_crime,trans_id,date,trans_type,operation,trans_ammount,balance
0,5314,1787,05-07-1993,96396,12,8033,-1,30,weekly issuance,22-03-1993,...,2985.0,2804,-181.0,5789.0,524055,21-05-1993,credit,credit in cash,5800.0,16800.0
1,5314,1787,05-07-1993,96396,12,8033,-1,30,weekly issuance,22-03-1993,...,2985.0,2804,-181.0,5789.0,524056,20-06-1993,credit,credit in cash,3300.0,20100.0
2,5316,1801,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,...,2854.0,2618,-236.0,5472.0,527456,28-02-1993,credit,credit in cash,18972.0,19674.9
3,5316,1801,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,...,2854.0,2618,-236.0,5472.0,528025,15-03-1993,credit,credit in cash,3700.0,23374.9
4,5316,1801,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,...,2854.0,2618,-236.0,5472.0,527446,16-03-1993,credit,credit in cash,8368.0,31742.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17788,7308,11362,27-12-1996,129408,24,5392,1,67,monthly issuance,14-10-1995,...,3244.0,3079,-165.0,6323.0,3424164,06-12-1996,withdrawal,remittance to another bank,129.0,39765.5
17789,7308,11362,27-12-1996,129408,24,5392,1,67,monthly issuance,14-10-1995,...,3244.0,3079,-165.0,6323.0,3424358,07-12-1996,withdrawal,withdrawal in cash,10400.0,29365.5
17790,7308,11362,27-12-1996,129408,24,5392,1,67,monthly issuance,14-10-1995,...,3244.0,3079,-165.0,6323.0,3424116,07-12-1996,withdrawal,remittance to another bank,330.0,29035.5
17791,7308,11362,27-12-1996,129408,24,5392,1,67,monthly issuance,14-10-1995,...,3244.0,3079,-165.0,6323.0,3424068,08-12-1996,withdrawal,remittance to another bank,56.0,28979.5


In [11]:
def normalize(df):

    columns = df.columns

    for col in columns:
        col_min = df[col].min()
        col_max =  df[col].max()
        
        df[col] = (df[col] - col_min)/\
                    (col_max - col_min)
            

In [21]:
f = utils.normalization(clients,"gender")
f = utils.normalization(f,"birth_number")
normalize(f)



,client_id,birth_number,gender,district_id
0,0.000000,0.418408,0.0,0.223684
1,0.000071,0.096897,1.0,0.000000
2,0.000143,0.281613,0.0,0.000000
3,0.000214,0.032088,1.0,0.052632
4,0.000286,0.080220,0.0,0.052632
...,...,...,...,...
5364,0.996928,0.972134,0.0,0.000000
5365,0.996999,0.163183,1.0,0.000000
5366,0.997857,0.396242,1.0,0.789474
5367,0.998071,0.607135,0.0,0.868421


In [65]:
### Full Normalization

cp = df.copy()
ct2 = clients.copy()

full_normalization(clients)


full_normalization(df)






,loan_id,account_id,loan_date,amount,duration,payments,status,district_id,frequency,account_date,...,no. of commited crimes '95,no. of commited crimes '96,crime_growth,total_crime,trans_id,date,trans_type,operation,trans_ammount,balance
0,0.116208,0.116208,0.177305,0.425397,0.00,0.959866,0.0,0.388889,1.0,0.726316,...,0.521127,0.450704,0.183099,0.507042,0.092907,0.671806,0.0,0.50,0.551202,0.002585
1,0.116208,0.116208,0.177305,0.425397,0.00,0.959866,0.0,0.388889,1.0,0.726316,...,0.521127,0.450704,0.183099,0.507042,0.092963,0.643172,0.0,0.50,0.496635,0.059449
2,0.119266,0.119266,0.382979,0.638095,0.50,0.622074,1.0,0.583333,0.5,0.403509,...,0.492958,0.394366,0.112676,0.408451,0.093132,0.890602,0.0,0.50,0.780529,0.049856
3,0.119266,0.119266,0.382979,0.638095,0.50,0.622074,1.0,0.583333,0.5,0.403509,...,0.492958,0.394366,0.112676,0.408451,0.093919,0.469163,0.0,0.50,0.504567,0.138139
4,0.119266,0.119266,0.382979,0.638095,0.50,0.622074,1.0,0.583333,0.5,0.403509,...,0.492958,0.394366,0.112676,0.408451,0.093019,0.501468,0.0,0.50,0.603365,0.383113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17788,1.000000,1.000000,0.872340,0.539683,0.25,0.698997,1.0,0.875000,0.5,0.466667,...,0.549296,0.521127,0.197183,0.549296,0.805025,0.193833,0.5,0.75,0.169471,0.603676
17789,1.000000,1.000000,0.872340,0.539683,0.25,0.698997,1.0,0.875000,0.5,0.466667,...,0.549296,0.521127,0.197183,0.549296,0.806879,0.228341,0.5,1.00,0.637740,0.310569
17790,1.000000,1.000000,0.872340,0.539683,0.25,0.698997,1.0,0.875000,0.5,0.466667,...,0.549296,0.521127,0.197183,0.549296,0.804463,0.228341,0.5,0.75,0.433654,0.299713
17791,1.000000,1.000000,0.872340,0.539683,0.25,0.698997,1.0,0.875000,0.5,0.466667,...,0.549296,0.521127,0.197183,0.549296,0.803901,0.261380,0.5,0.75,0.049760,0.297760


In [60]:
def full_normalization(df):
    
    cp = df.copy()
    columns = cp.columns

    for column in columns:
        if(cp[column].dtypes != 'int64' or cp[column].dtypes != 'float64'):
            cp = utils.normalization(cp,column)

    normalize(cp)
    return cp

In [ ]:
### Logistic Regression